In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pywt
import tkinter as tk
from tkinter import filedialog, Canvas, Scrollbar, OptionMenu, StringVar
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import csv
import time
from scipy import signal
# Global variables to keep track of file list and current index
data_folder = None
file_list = []
current_index = 0
signal_window = None
selected_rhythm = None
selected_morphology = None
canvas_widget = None
scrollbar = None
save_button = None
rhythm_menu = None
morphology_menu = None
signal_var = None
extension_menu = None
selected_extension = None  # Declare selected_extension as a global variable

# Function to create selected rhythm and morphology variables
def create_variables():
    global selected_rhythm, selected_morphology, selected_extension
    if selected_rhythm is None:
        selected_rhythm = StringVar()
    if selected_morphology is None:
        selected_morphology = StringVar()
    if selected_extension is None:  # Initialize selected_extension
        selected_extension = StringVar()

def create_save_button():
    global save_button
   # save_button = tk.Button(signal_window, text="Save", command=save_results)
   # save_button.grid(row=0, column=1, padx=10, pady=10)

def create_dropdown_menus():
    global rhythm_menu, morphology_menu, signal_var, extension_menu, selected_extension
    bottom_menu_frame = tk.Frame(root)
    bottom_menu_frame.grid(row=0, column=1, padx=10, pady=10)

    rhythm_label = tk.Label(bottom_menu_frame, text="Select Rhythm:")
    rhythm_label.grid(row=1, column=1)

    rhythm_options = ["Sinus Rhythm", "Atrial Fibrillation", "Atrial Flutter", "SVT", "VT"]
    rhythm_menu = OptionMenu(bottom_menu_frame, selected_rhythm, *rhythm_options)
    rhythm_menu.grid(row=1, column=2)

    morphology_label = tk.Label(bottom_menu_frame, text="Select Morphology:")
    morphology_label.grid(row=1, column=3)

    morphology_options = ["Normal", "LBBB", "RBBB", "NICD", "Ventricular"]
    morphology_menu = OptionMenu(bottom_menu_frame, selected_morphology, *morphology_options)
    morphology_menu.grid(row=1, column=4)

    signal_label = tk.Label(bottom_menu_frame, text="Select Signal:")
    signal_label.grid(row=1, column=5)

    signal_var = StringVar()
    signal_menu = OptionMenu(bottom_menu_frame, signal_var, *file_list, command=update_signal)
    signal_menu.grid(row=1, column=6)

def update_signal(selected_signal):
    global current_index
    current_index = file_list.index(selected_signal)
    show_current_signal()

def open_folder():
    global data_folder, file_list, current_index, save_button, extension_menu, selected_extension
    selected_extension_value = selected_extension.get() 
    new_data_folder = filedialog.askdirectory(title="Select Data Folder")
    if new_data_folder:
        
        data_folder = new_data_folder
        file_list = sorted([f for f in os.listdir(data_folder) if f.endswith(selected_extension_value)])
        current_index = 0
        create_variables()
        create_save_button()
        create_dropdown_menus()
        show_current_signal()


def apply_lowpass_filter(signal_data, sample_rate):
    nyquist_freq = 0.5 * sample_rate
    cutoff_freq = 150.0
    normalized_cutoff_freq = cutoff_freq / nyquist_freq
    b, a = signal.butter(4, normalized_cutoff_freq, btype='low', analog=False)
    filtered_signal = signal.lfilter(b, a, signal_data)
    return filtered_signal


def apply_power_line_filter(signal_data, sample_rate, power_line_frequency):
    nyquist_freq = 0.5 * sample_rate
    notch_freq = power_line_frequency / nyquist_freq
    quality_factor = 30.0  # Adjust the quality factor as needed
    b, a = signal.iirnotch(notch_freq, quality_factor)
    filtered_signal = signal.lfilter(b, a, signal_data)
    return filtered_signal

def show_current_signal():
    global current_index, canvas_widget, scrollbar, selected_rhythm, selected_morphology
    create_variables()

    if 0 <= current_index < len(file_list):
        file_path = os.path.join(data_folder, file_list[current_index])

        with open(file_path, 'rb') as fh:
            loaded_array = np.fromfile(fh, dtype=np.int16)

        sample_rate = 500

        num_leads = 12

        samples_per_lead = len(loaded_array) // num_leads

        start_time = 5.0
        end_time = start_time + 3.0

        start_sample = int(start_time * sample_rate)
        end_sample = int(end_time * sample_rate)

        if canvas_widget is not None:
            canvas_widget.get_tk_widget().destroy()
            scrollbar.destroy()

        canvas_width = 500
        canvas_height = 800  

        fig, axes = plt.subplots(7, 2, figsize=(17, 7 * 7), dpi=80)  # Adjust the grid to 7 rows

        subplot_height = 10
        fig.set_figheight(4 * 3)

        fig.subplots_adjust(hspace=0.2, wspace=0.1)
        lead_plots = []

        lead_names = ["I", "II", "III", "aVF", "aVR", "aVL", "V1", "V2", "V3", "V4", "V5", "V6"]

        for lead_index in range(num_leads):
            lead_data = loaded_array[lead_index::num_leads][start_sample:int(8.0 * sample_rate)]
            lead_time = np.linspace(start_time, 8.0, len(lead_data))

            
            filtered_lead_data = apply_lowpass_filter(lead_data, sample_rate)

            
            power_line_frequency = 60  
            filtered_lead_data = apply_power_line_filter(filtered_lead_data, sample_rate, power_line_frequency)

            column_index = lead_index % 2
            row_index = lead_index // 2

            
            min_y = -0.5
            max_y = 0.5
            normalized_lead_data = (filtered_lead_data - np.min(filtered_lead_data)) / (np.max(filtered_lead_data) - np.min(filtered_lead_data)) * (max_y - min_y) + min_y

           
            lead_plots.append(axes[row_index, column_index].plot(lead_time, normalized_lead_data, linestyle='-', linewidth=1.5, color='black'))
            axes[row_index, column_index].set_ylabel(f'{lead_names[lead_index]}', fontsize=12)

            axes[row_index, column_index].grid(axis='y', linestyle='-', linewidth=0.5)

           
            interval_x = 0.04  
            inter_x= 0.04
            num_lines_x = int((8 - start_time) / interval_x)
            for i in range(num_lines_x):
                time_value = start_time + i * interval_x
                axes[row_index, column_index].axvline(x=time_value, color='red', linestyle='-', linewidth=0.3)
                
           
            interval_y = 0.1  # mV
            min_y = -0.5
            max_y = 0.5
            num_lines_y = int((max_y - min_y) / interval_y)
            for i in range(num_lines_y):
                y_value = min_y + i * interval_y
                axes[row_index, column_index].axhline(y=y_value, color='red', linestyle='-', linewidth=0.3)

        axes[-1, 0].set_xlabel('Time (s)', fontsize=14)
        axes[-1, 1].set_xlabel('Time (s)', fontsize=14)

      
        v1_data = loaded_array[14::12][int(5.0 * sample_rate):int(10.0 * sample_rate)]  # Assuming V1 is the 7th lead (index 6)
        v1_time = np.linspace(5.0, 10.0, len(v1_data))

        
        v1_filtered_data = apply_lowpass_filter(v1_data, sample_rate)

       
        power_line_frequency = 50  
        v1_filtered_data = apply_power_line_filter(v1_filtered_data, sample_rate, power_line_frequency)

       
        v1_min_y = -0.5
        v1_max_y = 0.5
        v1_normalized_data = (v1_filtered_data - np.min(v1_filtered_data)) / (np.max(v1_filtered_data) - np.min(v1_filtered_data)) * (v1_max_y - v1_min_y) + v1_min_y

        # Create a new subplot for V1 at the bottom, spanning both columns
        v1_subplot = fig.add_subplot(7, 1, 7)
        v1_subplot.plot(v1_time, v1_normalized_data, linestyle='-', linewidth=1.5, color='black')
        v1_subplot.set_ylabel('V1', fontsize=12)
        v1_subplot.grid(axis='y', linestyle='-', linewidth=0.5)

        # Add vertical lines for V1
        for j in range(int((10.0 - 5.0) / inter_x)):
            time_value = 5.0 + j * inter_x
            v1_subplot.axvline(x=time_value, color='red', linestyle='-', linewidth=0.3)

        # Add horizontal lines for V1
        for j in range(num_lines_y):
            y_value = v1_min_y + j * interval_y
            v1_subplot.axhline(y=y_value, color='red', linestyle='-', linewidth=0.3)

        v1_subplot.set_xlabel('Time (s)', fontsize=14)

        # Create a new subplot for lead 1 at the bottom
        l1_data = loaded_array[0::12][int(5.0 * sample_rate):int(10.0 * sample_rate)]  # Assuming lead 1 is the first lead (index 0)
        l1_time = np.linspace(5.0, 10.0, len(l1_data))

        save_button.config(command=save_results)

        # Create a FigureCanvasTkAgg widget to display the Matplotlib figure in the Tkinter window
        canvas_widget = FigureCanvasTkAgg(fig, master=signal_window)
        canvas_widget.get_tk_widget().grid(row=0, column=0, sticky="nsew")

        # Create a vertical scrollbar
        scrollbar = Scrollbar(signal_window, orient=tk.VERTICAL)
        scrollbar.grid(row=0, column=1, sticky="ns")
        scrollbar.config(command=canvas_widget.get_tk_widget().yview)
        canvas_widget.get_tk_widget().config(yscrollcommand=scrollbar.set)

        canvas_widget.get_tk_widget().bind("<MouseWheel>", lambda event: canvas_widget.get_tk_widget().yview_scroll(int(-1*(event.delta/120)), "units"))

    else:
        print("No more signals to show.")
def show_next_signal():
    global current_index
    if current_index < len(file_list) - 1:
        current_index += 1
        show_current_signal()
    else:
        print("This is the last signal.")

def show_previous_signal():
    global current_index
    if current_index > 0:
        current_index -= 1
        show_current_signal()
    else:
        print("This is the first signal.")

# Function to save the selected rhythm and morphology to a CSV file
def save_results():
    global data_folder, selected_rhythm, selected_morphology
    if data_folder:
        if selected_rhythm.get() and selected_morphology.get():
            file_path = os.path.join(data_folder, "results.csv")
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

            if not os.path.exists(file_path):
                with open(file_path, mode='w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(["File Name", "Rhythm", "Morphology", "Timestamp"])

            with open(file_path, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([file_list[current_index], selected_rhythm.get(), selected_morphology.get(), timestamp])

# Create the main Tkinter window
root = tk.Tk()
root.title("ECG Signal Viewer")

# Create a dropdown menu for selecting the signal extension before opening the folder
save_button = tk.Button(root, text="Save", command=save_results)
save_button.grid(row=0, column=7, padx=10, pady=10)
    
extension_label = tk.Label(root, text="Select Signal Extension:")
extension_label.grid(row=0, column=8, padx=10, pady=10)

extension_options = [".D20", ".D21", ".D22", ".bin", ".ecg"]
selected_extension = StringVar()
extension_menu = OptionMenu(root, selected_extension, *extension_options)
extension_menu.grid(row=0, column=9, padx=10, pady=10)

# Create buttons using grid layout
open_button = tk.Button(root, text="Open Folder", command=open_folder)
open_button.grid(row=0, column=0, padx=10, pady=10)

prev_button = tk.Button(root, text="Previous", command=show_previous_signal)
prev_button.grid(row=0, column=10, padx=10, pady=10)

next_button = tk.Button(root, text="Next", command=show_next_signal)
next_button.grid(row=0, column=11, padx=10, pady=10)

# Create a frame to hold the signal display
signal_window = tk.Frame(root)
signal_window.grid(row=2, column=0, columnspan=13, sticky="nsew")

# Configure row and column weights for signal_window to make it expand with the window
root.grid_rowconfigure(2, weight=1)
root.grid_columnconfigure(0, weight=1)

# Start the Tkinter main loop
root.mainloop()
